In [59]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from PIL import Image
import cv2
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

In [60]:
csv_file = "../data/train_labels.csv"
csv_file_2 = "../data/train_labels_2.csv"

df_1 = pd.read_csv(csv_file)
df_2 = pd.read_csv(csv_file_2)
df = pd.concat([df_1, df_2], ignore_index=True, )
df.rename(columns={'Bags used': 'bags_used', 'Pothole number': 'pothole_number'}, inplace=True)


In [61]:
annotations_folder = "../data/train_annotations/"
annotations_folder_2 = "../data/train_annotations_2/"

In [62]:

# List to store the extracted data
data = []

# Loop through each annotation file
for file_name in os.listdir(annotations_folder):
    if file_name.endswith('.txt'):
        image_name = file_name.replace('.txt', '')  # Assuming image has the same name as the annotation file
        
        # Open and read the annotation file
        with open(os.path.join(annotations_folder, file_name), 'r') as file:
            for line in file:
                # Split the line into its components
                class_id, x_center, y_center, width, height = line.strip().split()
                
                # Append the data to the list
                data.append({
                    'image_name': image_name,
                    'class_id': int(class_id),
                    'x_center': float(x_center),
                    'y_center': float(y_center),
                    'width': float(width),
                    'height': float(height)
                })

# Convert the list to a DataFrame
anno = pd.DataFrame(data)


In [63]:
data = []

# Loop through each annotation file
for file_name in os.listdir(annotations_folder_2):
    if file_name.endswith('.txt'):
        image_name = file_name.replace('.txt', '')  # Assuming image has the same name as the annotation file
        
        # Open and read the annotation file
        with open(os.path.join(annotations_folder_2, file_name), 'r') as file:
            for line in file:
                # Split the line into its components
                class_id, x_center, y_center, width, height = line.strip().split()
                
                # Append the data to the list
                data.append({
                    'image_name': image_name,
                    'class_id': int(class_id),
                    'x_center': float(x_center),
                    'y_center': float(y_center),
                    'width': float(width),
                    'height': float(height)
                })

# Convert the list to a DataFrame
anno_2 = pd.DataFrame(data)

# Display the DataFrame
print(anno_2.head())

  image_name  class_id  x_center  y_center     width    height
0       p930         0  0.449839  0.499142  0.799943  0.998284
1       p930         1  0.550746  0.162950  0.071658  0.325900
2       p924         1  0.777287  0.467410  0.445397  0.049743
3       p924         0  0.519282  0.546312  0.917953  0.704974
4        p58         0  0.481026  0.474271  0.483542  0.667238


In [64]:
annotations = pd.concat([anno, anno_2], ignore_index=True, )
annotations['pothole_number'] = annotations['image_name'].str.replace('p', '').astype(int)
annotations

,image_name,class_id,x_center,y_center,width,height,pothole_number
0,p1336,1,0.634134,0.553173,0.562607,0.094340,1336
1,p1108,0,0.540652,0.533448,0.560892,0.463122,1108
2,p1108,1,0.576672,0.540309,0.564322,0.174957,1108
3,p1120,1,0.387135,0.484563,0.596913,0.053173,1120
4,p1120,0,0.439451,0.510292,0.372213,0.313894,1120
...,...,...,...,...,...,...,...
1716,p1663,1,0.455288,0.472034,0.727885,0.073593,1663
1717,p55,0,0.461758,0.451115,0.923516,0.356775,55
1718,p55,1,0.190507,0.435678,0.334832,0.027444,55
1719,p929,0,0.528169,0.550600,0.782471,0.874786,929


In [65]:
df[df['pothole_number'] == 929]

,pothole_number,Bags used
263,929,1.0
833,929,1.0


In [66]:
df = df.drop_duplicates()
df_annotations = annotations.merge(df,on='pothole_number', how="inner", suffixes=('', '_df'))
df_annotations['img_file'] = 'p' + df_annotations['pothole_number'].astype(str) + '.jpg'

In [67]:
df_annotations

,image_name,class_id,x_center,y_center,width,height,pothole_number,Bags used,img_file
0,p1336,1,0.634134,0.553173,0.562607,0.094340,1336,0.25,p1336.jpg
1,p1108,0,0.540652,0.533448,0.560892,0.463122,1108,0.50,p1108.jpg
2,p1108,1,0.576672,0.540309,0.564322,0.174957,1108,0.50,p1108.jpg
3,p1120,1,0.387135,0.484563,0.596913,0.053173,1120,0.25,p1120.jpg
4,p1120,0,0.439451,0.510292,0.372213,0.313894,1120,0.25,p1120.jpg
...,...,...,...,...,...,...,...,...,...
1514,p1663,1,0.455288,0.472034,0.727885,0.073593,1663,1.00,p1663.jpg
1515,p55,0,0.461758,0.451115,0.923516,0.356775,55,2.00,p55.jpg
1516,p55,1,0.190507,0.435678,0.334832,0.027444,55,2.00,p55.jpg
1517,p929,0,0.528169,0.550600,0.782471,0.874786,929,1.00,p929.jpg


In [68]:
valid = [] 
for file in df_annotations['img_file']:
    name = '../data/train_images/' + file 
    name_2 = '../data/train_images_2/' + file 
    if os.path.exists(name):
        valid.append(file)
    elif os.path.exists(name_2):
        valid.append(file)

In [69]:
len(valid)

1519

In [70]:
valid_df = df_annotations[df_annotations['img_file'].isin(valid)]
valid_df

,image_name,class_id,x_center,y_center,width,height,pothole_number,Bags used,img_file
0,p1336,1,0.634134,0.553173,0.562607,0.094340,1336,0.25,p1336.jpg
1,p1108,0,0.540652,0.533448,0.560892,0.463122,1108,0.50,p1108.jpg
2,p1108,1,0.576672,0.540309,0.564322,0.174957,1108,0.50,p1108.jpg
3,p1120,1,0.387135,0.484563,0.596913,0.053173,1120,0.25,p1120.jpg
4,p1120,0,0.439451,0.510292,0.372213,0.313894,1120,0.25,p1120.jpg
...,...,...,...,...,...,...,...,...,...
1514,p1663,1,0.455288,0.472034,0.727885,0.073593,1663,1.00,p1663.jpg
1515,p55,0,0.461758,0.451115,0.923516,0.356775,55,2.00,p55.jpg
1516,p55,1,0.190507,0.435678,0.334832,0.027444,55,2.00,p55.jpg
1517,p929,0,0.528169,0.550600,0.782471,0.874786,929,1.00,p929.jpg


In [71]:
def yolo_to_bbox(x_center, y_center, width, height, img_width, img_height):
    x_min = (x_center - width / 2) * img_width
    x_max = (x_center + width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    y_max = (y_center + height / 2) * img_height
    return x_min, y_min, x_max, y_max

# Calculate the diagonal length of the bounding box for `class_id` 1
def calculate_stick_size(row, img_width, img_height):
    if (row['class_id'] != 1):
        return 0
    x_min, y_min, x_max, y_max = yolo_to_bbox(
        row['x_center'], row['y_center'], row['width'], row['height'], img_width, img_height
    )
    return np.sqrt((x_max - x_min) ** 2 + (y_max - y_min) ** 2)

# Example function to load an image and get its dimensions
def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size

# Add a column with stick sizes
df_annotations['stick_size'] = df_annotations.apply(
    lambda row: calculate_stick_size(row, *get_image_size(f'../data/train_all/{row["img_file"]}')),
    axis=1
)

In [72]:
df_annotations['stick_size'] = df_annotations.groupby('pothole_number')['stick_size'].transform(
    lambda x: x.replace(0, x.max())
)

# If stick_size is still zero, populate it with the mean of non-zero stick_size values
mean_stick_size = df_annotations['stick_size'].replace(0, np.nan).mean()
df_annotations['stick_size'] = df_annotations['stick_size'].replace(0, mean_stick_size)



In [73]:
df_annotations['has_pothole'] = df_annotations.groupby('pothole_number')['class_id'].transform(lambda x: (x == 0).any())


In [74]:
df_annotations = df_annotations.reset_index(drop=True)
df_annotations['stick_size'].idxmax()


1275

In [75]:
idx_max = df_annotations['stick_size'].idxmax()
if idx_max in df_annotations.index:
    row_max = df_annotations.loc[idx_max]
    print(row_max)
else:
    print(f"Index {idx_max} not found in DataFrame.")

image_name             p1601
class_id                   0
x_center            0.427885
y_center                 0.5
width               0.848077
height              0.997321
pothole_number          1601
Bags used                0.5
img_file           p1601.jpg
stick_size        842.645863
has_pothole             True
Name: 1275, dtype: object


In [76]:
stick_min = df_annotations['stick_size'].mean()
df_annotations['scale_factor'] = (400 / df_annotations['stick_size'])

# Convert scale_factor to integer type if needed
#df_annotations['scale_factor'] = df_annotations['scale_factor'].astype(int)
df_annotations

,image_name,class_id,x_center,y_center,width,height,pothole_number,Bags used,img_file,stick_size,has_pothole,scale_factor
0,p1336,1,0.634134,0.553173,0.562607,0.094340,1336,0.25,p1336.jpg,215.634563,False,1.854990
1,p1108,0,0.540652,0.533448,0.560892,0.463122,1108,0.50,p1108.jpg,223.330280,True,1.791069
2,p1108,1,0.576672,0.540309,0.564322,0.174957,1108,0.50,p1108.jpg,223.330280,True,1.791069
3,p1120,1,0.387135,0.484563,0.596913,0.053173,1120,0.25,p1120.jpg,226.526572,True,1.765797
4,p1120,0,0.439451,0.510292,0.372213,0.313894,1120,0.25,p1120.jpg,226.526572,True,1.765797
...,...,...,...,...,...,...,...,...,...,...,...,...
1514,p1663,1,0.455288,0.472034,0.727885,0.073593,1663,1.00,p1663.jpg,690.079562,True,0.579643
1515,p55,0,0.461758,0.451115,0.923516,0.356775,55,2.00,p55.jpg,117.200459,True,3.412956
1516,p55,1,0.190507,0.435678,0.334832,0.027444,55,2.00,p55.jpg,117.200459,True,3.412956
1517,p929,0,0.528169,0.550600,0.782471,0.874786,929,1.00,p929.jpg,309.195260,True,1.293681


In [77]:
def resize_image(image_path, scale_factor, output_path):
    with Image.open(image_path) as img:
        # Calculate new size
        width, height = img.size
        new_size = (int(width * scale_factor), int(height * scale_factor))
        
        # Resize and save image
        img_resized = img.resize(new_size, Image.LANCZOS)
        img_resized.save(output_path)  # Save resized image

# Define a target size for the stick
for index, row in df_annotations.iterrows():
    image_path = f"../data/train_all/{row['img_file']}"
    output_path = f"../data/cropped_images/{row['img_file']}"
    scale_factor = row['scale_factor']
    
    resize_image(image_path, scale_factor, output_path)

# Resize all images to match the target size of the stick



In [78]:
def resize_and_pad_image(image_path, output_path, target_size=(512, 512), max_image_size=None):
    with Image.open(image_path) as img:
        # Calculate scaling factor based on the largest image
        if max_image_size is None:
            max_image_size = max(img.size)
        
        scale_factor = target_size[0] / max_image_size
        
        # Resize the image while keeping aspect ratio
        new_size = (int(img.width * scale_factor), int(img.height * scale_factor))
        img_resized = img.resize(new_size, Image.LANCZOS)
        
        # Create a new image with a black background
        new_img = Image.new("RGB", target_size, (0, 0, 0))
        
        # Calculate padding
        padding_x = (target_size[0] - new_size[0]) // 2
        padding_y = (target_size[1] - new_size[1]) // 2
        
        # Paste the resized image onto the black background
        new_img.paste(img_resized, (padding_x, padding_y))
        
        # Save the new image
        new_img.save(output_path)

# Example usage
image_paths = []
for index, row in df_annotations.iterrows():
    image_paths.append(f"../data/cropped_images/{row['img_file']}")
output_folder = "../data/padded_images/"
os.makedirs(output_folder, exist_ok=True)

# Find the largest image size
max_image_size = max([max(Image.open(image_path).size) for image_path in image_paths])
# Resize and pad each image
for image_path in image_paths:
    output_path = os.path.join(output_folder, f"resized_{os.path.basename(image_path)}")
    resize_and_pad_image(image_path, output_path, target_size=(512, 512), max_image_size=max_image_size)


In [136]:
scale_factor

1.2936808920974263

In [125]:
def add_randomness_to_colors_tf(image):
    color_shift_range = tf.random.uniform([], 0, 15, dtype=tf.float32)

    # Convert the image to a float32 tensor and normalize the values to [0, 1]
    img_tensor = tf.image.convert_image_dtype(image, tf.float32)

    # Generate random shifts for each color channel
    random_shift = tf.random.uniform(tf.shape(img_tensor), -color_shift_range/255.0, color_shift_range/255.0)

    # Add the random shift and clip values to stay within valid range [0, 1]
    img_tensor = tf.clip_by_value(img_tensor + random_shift, 0.0, 1.0)

    # Convert back to the original data type
    img_tensor = tf.image.convert_image_dtype(img_tensor, tf.float32)

    return img_tensor
class Augment(tf.keras.layers.Layer):
    def __init__(self, contrast_range=[0.4, 1.5], 
                 brightness_delta=[-0.4, 0.3],
                 hue_delta=[-0.1, 0.1],
                 jpeg_qual = [40,100],
                 **kwargs):
        super(Augment, self).__init__(**kwargs)
        self.contrast_range = contrast_range
        self.brightness_delta = brightness_delta
        self.hue_delta = hue_delta
        self.jpeg_qual = jpeg_qual
    
    def ensure_rank_4(self, images):
        """Ensure that the images tensor has rank 4."""
        if len(images.shape) == 3:
            # If the image is rank 3 (height, width, channels), add a batch dimension
            images = tf.expand_dims(images, axis=0)
        elif len(images.shape) == 5:
            # If the image is rank 5 (batch_size, height, width, channels, extra), squeeze out the extra dimension
            images = tf.squeeze(images, axis=-1)
        return images
    
    def call(self, images, training=None):
        if not training:
            return images
        
        images = self.ensure_rank_4(images)
        
        contrast = np.random.uniform(
            self.contrast_range[0], self.contrast_range[1])
        brightness = np.random.uniform(
            self.brightness_delta[0], self.brightness_delta[1])
        hue = np.random.uniform(
            self.hue_delta[0], self.hue_delta[1])
        jpeg = random.randint(
            self.jpeg_qual[0], self.jpeg_qual[1])
        flip_u = random.randint(0,2)
        flip_l = random.randint(0,2)
        
        contr = random.randint(0,3)
        bright = random.randint(0,3)

        ad_hue = random.randint(0,3)
        j_q = random.randint(0,2)

        rand_col = random.randint(0,4)
        soft = random.randint(0,2)

        rot = random.randint(0, 3)

        if (contr == 1) :images = tf.image.adjust_contrast(images, contrast)
        if (bright == 1) :images = tf.image.adjust_brightness(images, brightness)
        images = tf.clip_by_value(images, 0, 1)
        if (ad_hue == 1) :images = tf.image.adjust_hue(images, hue)
        #if (j_q== 1) :images = tf.image.adjust_jpeg_quality(images, jpeg, dct_method='')
        if (rand_col == 1) :images = add_randomness_to_colors_tf(images)
        #if (soft == 1) :images = soften_edges_tf(images)
        if (flip_l == 1) : images = tf.image.flip_left_right(images)
        if (flip_u == 1) : images = tf.image.flip_up_down(images)
        if (rot == 1) : images =tf.image.rot90(images)

        return images


In [126]:

# Define image dimensions
img_width, img_height = 256, 256

# Preprocess images and load into arrays
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0, 1]
    return img_array

# Load and preprocess images
X = np.array([preprocess_image('../data/padded_images/resized_' + file) for file in valid_df['img_file']])
y = valid_df['Bags used '].values
scaler = MinMaxScaler()

# Split data into train and test sets
X_train, X_test, y_train_un, y_test_un = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = scaler.fit_transform(y_train_un.reshape(-1, 1)).flatten()
y_test = scaler.transform(y_test_un.reshape(-1, 1)).flatten()


In [127]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    patience=50,          # Stop after 5 epochs of no improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best loss
)

In [128]:
augment_layer = Augment()

model = Sequential([
    InputLayer(input_shape=(256, 256, 3)),  # Replace with your input image size
    augment_layer,  # Add the augmentation layer here
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Regression output for predicting 'bags_used'
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Summary of the model
model.summary()


/Users/steffanschoonbee/Documents/Studies/Hackathon/StandardBankHackathon/notebook/venv/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augment_30 (Augment)            │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 125, 125, 16)   │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 62, 62, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 61504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_99 (Dense)                │ (None, 32)             │     1,968,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_70 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_100 (Dense)               │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_71 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_101 (Dense)               │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,974,225 (7.53 MB)

 Trainable params: 1,974,225 (7.53 MB)

 Non-trainable params: 0 (0.00 B)

In [129]:
history = model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test), batch_size=15, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')

Epoch 1/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 12s 137ms/step - loss: 0.0548 - mean_absolute_error: 0.0998 - val_loss: 0.0030 - val_mean_absolute_error: 0.0341
Epoch 2/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 11s 140ms/step - loss: 0.0078 - mean_absolute_error: 0.0404 - val_loss: 0.0033 - val_mean_absolute_error: 0.0363
Epoch 3/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 12s 145ms/step - loss: 0.0081 - mean_absolute_error: 0.0448 - val_loss: 0.0028 - val_mean_absolute_error: 0.0336
Epoch 4/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 12s 146ms/step - loss: 0.0065 - mean_absolute_error: 0.0398 - val_loss: 0.0033 - val_mean_absolute_error: 0.0367
Epoch 5/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 11s 134ms/step - loss: 0.0094 - mean_absolute_error: 0.0458 - val_loss: 0.0032 - val_mean_absolute_error: 0.0349
Epoch 6/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 11s 140ms/step - loss: 0.0078 - mean_absolute_error: 0.0423 - val_loss: 0.0032 - val_mean_absolute_error: 0.0369
Epoch 7/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 11s 133ms/step - loss: 0.0048 - mean_absolute_err

In [131]:
# Save the weights
#model.save('../model_weights/CNN_padded_images.keras')


In [130]:
metric = tf.metrics.R2Score()
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R2 Score: ", r2)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step
R2 Score:  0.5094366778419466


In [138]:
input_img = load_img("../data/padded_images/resized_p39.jpg")

img_width, img_height = 256, 256
input_img = input_img.resize((img_width, img_height))
img_array = img_to_array(input_img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)
prediction = model.predict(img_array)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
[[0.03780432]]


In [133]:
# Assuming y_true are the actual values and y_pred are the predictions from your model

def round_to_nearest_25(values):
    rounded_values = np.round(values * 4) / 4
    return rounded_values

metric = tf.metrics.R2Score()
y_pred = model.predict(X_test)
y_in = scaler.inverse_transform(y_pred)
y_rounded = round_to_nearest_25(y_in)
r2 = r2_score(y_test_un, y_rounded)
r2

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step


0.5121133270477398

In [110]:

def round_to_nearest_25(values):
    rounded_values = np.round(values * 4) / 4
    return rounded_values

metric = tf.metrics.R2Score()
y_pred = model.predict(X_test)
y_in = scaler.inverse_transform(y_pred)
y_rounded = round_to_nearest_25(y_in)
r2 = r2_score(y_test_un, y_rounded)
r2

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step


0.12729084372709687

In [111]:
loaded_model = tf.keras.models.load_model('../model_weights/CNN_padded_images.keras', custom_objects={'Augment': Augment})
loaded_model.summary()

/Users/steffanschoonbee/Documents/Studies/Hackathon/StandardBankHackathon/notebook/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:562: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augment_14 (Augment)            │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_31 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 125, 125, 16)   │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ (None, 62, 62, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 61504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 32)             │     1,968,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,948,452 (15.06 MB)

 Trainable params: 1,974,225 (7.53 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,974,227 (7.53 MB)

In [31]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

In [54]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-7:]:
    layer.trainable = True


augment_layer = Augment()
# Input layer
inputs = Input(shape=(256, 256, 3))

# Add the augment layer
x = augment_layer(inputs)

# Pass the augmented images through the ResNet model
x = base_model(x)

# Add global pooling and custom layers
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)

# Create the final model
model = Model(inputs=inputs, outputs=predictions)
model.summary()

model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mean_absolute_error'])


early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    patience=20,          # Stop after 5 epochs of no improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best loss
)




Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_32 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ augment_16 (Augment)            │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,721,473 (90.49 MB)

 Trainable params: 3,549,313 (13.54 MB)

 Non-trainable params: 20,172,160 (76.95 MB)

In [55]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=15,
    callbacks=[early_stopping]
)

Epoch 1/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 52s 618ms/step - loss: 0.1959 - mean_absolute_error: 0.2686 - val_loss: 0.0081 - val_mean_absolute_error: 0.0822
Epoch 2/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 49s 603ms/step - loss: 0.0094 - mean_absolute_error: 0.0459 - val_loss: 0.0049 - val_mean_absolute_error: 0.0588
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 47s 584ms/step - loss: 0.0080 - mean_absolute_error: 0.0408 - val_loss: 0.0045 - val_mean_absolute_error: 0.0405
Epoch 4/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 47s 582ms/step - loss: 0.0129 - mean_absolute_error: 0.0439 - val_loss: 0.0055 - val_mean_absolute_error: 0.0460
Epoch 5/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 48s 588ms/step - loss: 0.0109 - mean_absolute_error: 0.0462 - val_loss: 0.0036 - val_mean_absolute_error: 0.0355
Epoch 6/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 48s 588ms/step - loss: 0.0112 - mean_absolute_error: 0.0440 - val_loss: 0.0035 - val_mean_absolute_error: 0.0376
Epoch 7/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 49s 602ms/step - loss: 0.0086 - mean_absolute_error: 0.0

In [57]:
metric = tf.metrics.R2Score()
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R2 Score: ", r2)

10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 890ms/step
R2 Score:  0.10469072408692237


In [60]:
X = np.array([preprocess_image('../data/padded_images/resized_' + file) for file in valid_df['img_file']])
y = valid_df['Bags used '].values
file_names = valid_df['img_file'].values  # Get the file names

# Split data into train and test sets
X_train, X_test, y_train_un, y_test_un, train_files, test_files = train_test_split(X, y, file_names, test_size=0.2, random_state=42)


Test file names: ['p1156.jpg' 'p154.jpg' 'p245.jpg' 'p928.jpg' 'p1121.jpg' 'p1655.jpg'
 'p1038.jpg' 'p963.jpg' 'p1515.jpg' 'p294.jpg' 'p1080.jpg' 'p1248.jpg'
 'p1264.jpg' 'p1232.jpg' 'p1586.jpg' 'p420.jpg' 'p39.jpg' 'p423.jpg'
 'p1291.jpg' 'p460.jpg' 'p1267.jpg' 'p135.jpg' 'p284.jpg' 'p1613.jpg'
 'p1212.jpg' 'p928.jpg' 'p974.jpg' 'p1081.jpg' 'p454.jpg' 'p422.jpg'
 'p1144.jpg' 'p124.jpg' 'p1246.jpg' 'p1571.jpg' 'p69.jpg' 'p1179.jpg'
 'p1602.jpg' 'p972.jpg' 'p426.jpg' 'p1577.jpg' 'p1168.jpg' 'p52.jpg'
 'p1271.jpg' 'p1049.jpg' 'p943.jpg' 'p499.jpg' 'p1563.jpg' 'p996.jpg'
 'p291.jpg' 'p940.jpg' 'p1527.jpg' 'p271.jpg' 'p979.jpg' 'p456.jpg'
 'p1619.jpg' 'p192.jpg' 'p498.jpg' 'p476.jpg' 'p1586.jpg' 'p1140.jpg'
 'p1551.jpg' 'p1560.jpg' 'p1017.jpg' 'p278.jpg' 'p1259.jpg' 'p1667.jpg'
 'p276.jpg' 'p1190.jpg' 'p197.jpg' 'p283.jpg' 'p1049.jpg' 'p1344.jpg'
 'p184.jpg' 'p1123.jpg' 'p1187.jpg' 'p935.jpg' 'p932.jpg' 'p933.jpg'
 'p174.jpg' 'p1588.jpg' 'p296.jpg' 'p949.jpg' 'p1098.jpg' 'p439.jpg'
 'p1568

In [118]:
input_img = load_img("../data/padded_images/resized_p142.jpg")

img_width, img_height = 256, 256
input_img = input_img.resize((img_width, img_height))
img_array = img_to_array(input_img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)
loaded_model = tf.keras.models.load_model('../model_weights/CNN_padded_images.keras', custom_objects={'Augment': Augment})
prediction = loaded_model.predict(img_array)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[[0.03983825]]
